# Application Manager

In [ ]:
from time import sleep
import requests
import ipywidgets as ipw
from os import path
import subprocess
from collections import OrderedDict
from IPython.display import clear_output

In [ ]:
# load registry
registry_url = 'https://raw.githubusercontent.com/aiidalab/aiidalab-registry/master/apps.json'
registry = requests.get(registry_url).json()
from config import aiidalab_apps

In [ ]:
def get_appdir(name):
    return path.join(aiidalab_apps, name)

In [ ]:
def update_app_list():
    selected = app_select.value
    app_select.value = None
    options = OrderedDict()
    for name in sorted(registry.keys()):
        label = registry[name]['label']
        if path.exists(get_appdir(name)):
            label += " (installed)"
        options[label] = name
    app_select.options = options
    app_select.value = selected

In [ ]:
def on_app_change(c):
    with out_app:
        clear_output()
        if app_select.value:
            display_app_box(app_select.value)

app_select = ipw.Select(layout={'height': '%dpx'%(len(registry)*20)})
app_select.observe(on_app_change, names='value')

out_app = ipw.Output()

btn_check_all = ipw.Button(description='Check for updates.')
btn_check_all.on_click(lambda b: check_all_updates())

display(ipw.VBox([app_select, btn_check_all, ipw.HTML('<hr>'), out_app, ipw.HTML('<hr>')]))
update_app_list()

In [ ]:
def display_app_box(name):
    print("Name: "+name)
    git_url = registry[name]['git_url']
    print("URL: "+git_url)
    meta_url = registry[name]['meta_url']
    metadata = requests.get(meta_url).json()
    print("Description: "+metadata['description'])
    
    if not path.exists(get_appdir(name)):
        btn = ipw.Button(description="install")
        btn.on_click(lambda b: install_app(name))   
        display(btn)
    else:
        btn_remove = ipw.Button(description="remove")
        btn_remove.disabled = (name == "home")
        btn_remove.on_click(lambda b: remove_app(name))
        
        btn_do_update = ipw.Button(description='Update app', disabled=True)
        btn_do_update.disabled = (check_update(name) != 'update')
        btn_do_update.on_click(lambda b: install_update(name))
        
        ref_drop = ipw.Dropdown(description="Version:")

        display(ipw.HBox([btn_remove, btn_do_update, ref_drop]))

        ver_sel = mk_version_selector(name) # move this down so the UI is more responsive
        ref_drop.options=ver_sel['options']
        ref_drop.value=ver_sel['head']
        def on_ref_change(c):
            change_ref(name, ref_drop.value)
        ref_drop.observe(on_ref_change, names='value')

In [ ]:
def check_all_updates():
    with out_app:
        display(ipw.HTML('<hr>'))
        print 'Looking for updates...'
        def check_all():
            res = {}
            for name in sorted(registry.keys()):
                apppath = get_appdir(name)
                if path.exists(apppath):
                    res[name] = check_update(name)
            return res

        r = []
        for name, u in check_all().items():
            if u == 'update':
                r.append(name)
            # TODO: elif u == 'conflict':

        def on_update_all(r):
            for u in r:
                install_update(u)

        if len(r) > 0:
            print 'Updates available for {}.'.format(', '.join(r))
            btn_update_all = ipw.Button(description='Update all.')
            btn_update_all.on_click(lambda b: on_update_all(r))
            display(btn_update_all)
        else:
            print 'No updates available.'
            
def check_update(name):
    appdir = get_appdir(name)
    git_fetch = ! git -C $appdir fetch
    git_status = ! git -C $appdir status
    git_head_hash = ! git -C $appdir rev-parse HEAD
    git_latest_hash = ! git -C $appdir rev-parse origin/master

    # returns ['0'] if the latest commit is an ancestor of HEAD
    git_head_hash = git_head_hash[0]
    git_latest_hash = git_latest_hash[0]
    git_is_ancestor = !git -C $appdir merge-base --is-ancestor $git_head_hash $git_latest_hash; echo $?
        
    if git_is_ancestor[0] == '0' and 'Your branch is behind' in git_status[1]:
        if 'and can be fast-forwarded' in git_status[1]:
            return 'update'
        else:
            return 'conflict'
    else:
        return 'no update'
    
def install_update(name):
    with out_app:
        print('Updating app {}...'.format(name))
        appdir = get_appdir(name)
        git_pull = ! git -C $appdir pull
        print git_pull
        print('Update done.')
        sleep(1)
    update_app_list()

In [ ]:
def mk_version_selector(name):
    git_url = registry[name]['git_url']
    appdir = get_appdir(name)
    ! git -C $appdir fetch --prune --quiet
    
    
    ref_lines = ! git -C $appdir show-ref
    options = {}
    for line in ref_lines:
        commit, ref = line.split()
        options[ref] = commit

    head_lines = ! git -C $appdir rev-parse HEAD
    head = head_lines[0]
    if head not in options.values():
        options[head] = head
    #html = ipw.HTML("<pre>head: {}</pre><pre>appdir: {}</pre>".format(head, appdir))
    
    return {'options': options,
            'head': head}

In [ ]:
def change_ref(name, commit):
    print("Changing version of app %s..."%name)
    appdir = get_appdir(name)
    ! git -C $appdir checkout --quiet $commit
    sleep(1)
    update_app_list()

In [ ]:
def install_app(name):
    print("Installing app %s..."%name)
    assert name != "home"
    git_url = registry[name]["git_url"]
    appdir = get_appdir(name)
    ! git clone --quiet $git_url $appdir
    #TODO maybe call "jupyter trust" on all notebooks
    sleep(1)
    update_app_list()

In [ ]:
def remove_app(name):
    print("Removing app %s..."%name)
    assert name != "home"
    appdir = get_appdir(name)
    ! rm -rf $appdir
    sleep(1)
    update_app_list()

In [ ]:
%%html
<a href="./start.ipynb"><button>Return to Launcher</button></a>

In [ ]:
if app_select.value is None:
    app_select.value = 'home'